In [57]:
import pandas as pd
import re

from .mappings import PROVINCIAL_MAPPINGS

RAW_DATA_DIRECTORY = "raw/poverty"
FINAL_DATA_DIRECTORY = "final/poverty"

ImportError: attempted relative import with no known parent package

In [59]:
def get():
    food_threshold_subsistence_families = pd.read_csv(f"{RAW_DATA_DIRECTORY}/food_threshold_subsistence_families.csv", skiprows=2, sep=";")

    food_threshold_subsistence_families["Geolocation"] = food_threshold_subsistence_families["Geolocation"].str\
                            .replace(r'^\.+', "", regex=True)\
                            .replace(r',', "", regex=True)\
                            .replace(r'(\w|\d)/', "", regex=True)\
                            .str.strip()

    food_threshold_subsistence_families["ADM2_CODE"] = food_threshold_subsistence_families["Geolocation"].apply(lambda x: PROVINCIAL_MAPPINGS[x])

    irrelevant_columns = [
        "Coefficient of Variation 2015",
        "Coefficient of Variation 2018",
        "Coefficient of Variation 2021p",
        "Standard Error 2015",
        "Standard Error 2018",
        "Standard Error 2021p",
        "90% Confidence Interval (Lower Limit) 2015",
        "90% Confidence Interval (Lower Limit) 2018",
        "90% Confidence Interval (Lower Limit) 2021p",
        "90% Confidence Interval (Upper Limit) 2015",
        "90% Confidence Interval (Upper Limit) 2018",
        "90% Confidence Interval (Upper Limit) 2021p",
    ]

    rename_columns = {
            "Annual Per Capita Food Threshold (in PhP) 2015":"Per Capita Food Threshold 2015",
            "Annual Per Capita Food Threshold (in PhP) 2018":"Per Capita Food Threshold 2018",
            "Annual Per Capita Food Threshold (in PhP) 2021p":"Per Capita Food Threshold 2021p",
            "Subsistence Incidence among Families (%) 2015":"Subsistence Incidence 2015",
            "Subsistence Incidence among Families (%) 2018":"Subsistence Incidence 2018",
            "Subsistence Incidence among Families (%) 2021p":"Subsistence Incidence 2021p",
    }

    food_threshold_subsistence_families = food_threshold_subsistence_families.drop(columns=irrelevant_columns)\
                                                    .rename(columns=rename_columns)
    
    food_threshold_subsistence_families = food_threshold_subsistence_families.melt(id_vars=["ADM2_CODE", "Geolocation"], var_name="Attribute Year", value_name="Value")
    food_threshold_subsistence_families["Year"] = food_threshold_subsistence_families["Attribute Year"].str.split(" ").str[-1]
    food_threshold_subsistence_families["Attribute"] = food_threshold_subsistence_families["Attribute Year"].str.split(" ").str[:-1].str.join(" ")
    del food_threshold_subsistence_families["Attribute Year"]
    food_threshold_subsistence_families = food_threshold_subsistence_families.pivot_table(index=["ADM2_CODE", "Geolocation", "Attribute"], columns="Year", values="Value", aggfunc='sum').reset_index()

    food_threshold_subsistence_families.to_csv(f"{FINAL_DATA_DIRECTORY}/food_threshold_subsistence_families.csv", index=False)
    return food_threshold_subsistence_families       

get()

Year,ADM2_CODE,Geolocation,Attribute,2015,2018,2021p
0,PH012800000,Ilocos Norte,Per Capita Food Threshold,16414,18623,22128
1,PH012800000,Ilocos Norte,Subsistence Incidence,0.3,0.1,0.0
2,PH012900000,Ilocos Sur,Per Capita Food Threshold,16449,17240,22852
3,PH012900000,Ilocos Sur,Subsistence Incidence,1.9,0.6,3.4
4,PH013300000,La Union,Per Capita Food Threshold,15282,15651,19547
...,...,...,...,...,...,...
169,PH175200000,Oriental Mindoro,Subsistence Incidence,6.8,1.9,5.4
170,PH175300000,Palawan,Per Capita Food Threshold,13740,16308,16788
171,PH175300000,Palawan,Subsistence Incidence,2.2,1.8,2.0
172,PH175900000,Romblon,Per Capita Food Threshold,15253,16794,20122
